In [ ]:
from hydromt_wflow import WflowModel
from hydromt_sfincs import SfincsModel
import xarray as xr
from os.path import join, isfile


In [ ]:
mod_sfincs = SfincsModel(r'../../3_models/sfincs/00_base', mode='r')
gdf_src = mod_sfincs.staticgeoms['src_snapped']

In [ ]:
postfix = '_vito_ksath1000' #'_vito_chirps'
mod = WflowModel(r'../../3_models/wflow', config_fn=f'wflow_sbm{postfix}.toml', mode='r')
ds_src = mod.staticmaps.raster.sample(gdf_src).load()
fn_q = join(mod.root, mod.config['output']['path'].replace('.nc', '_src.nc'))
if isfile(fn_q):
    da_q = xr.open_dataarray(fn_q)
else:
    da_q = mod.results['output']['q_river'].raster.sample(gdf_src)
    da_q.to_netcdf(fn_q)

In [ ]:
from eva import plot_return_values, eva_block_maxima
import matplotlib.pyplot as plt
period='AS-AUG'

ds_q_eva = eva_block_maxima(
    da_q,
    period = period,
    min_dist = 14,
)

ds0 = ds_q_eva.isel(index=0)
plot_return_values(
    ds0['peaks'],
    ds0['parameters'],
    ds0['distribution'].item(),
)
fn_fig = join(mod.root, f'eva_index{ds0.index.item()}{postfix}.png')
plt.savefig(fn_fig, bbox_inches='tight', dpi=225)

In [ ]:
events = {
    'idai': ('20190310', '20190330'),
    'eloise': ('20210110', '20210131')
}
for name, dates in events.items():
    da_q.sel(time=slice(*dates)).to_series().unstack().plot()
    fn_fig = join(mod.root, f'{name}{postfix}.png')
    plt.savefig(fn_fig, bbox_inches='tight', dpi=225)

In [ ]:
rm = {'x_out': 'x', 'y_out': 'y', 'wflow_uparea': 'uparea'}
gdf_q2 = xr.merge([
    ds_src[rm.keys()].rename(rm).reset_coords(drop=True),
    ds_q_eva['return_values'].sel(rps=2).rename('Q2').reset_coords(drop=True),
]).vector.to_gdf()
gdf_q2.set_crs(ds_src.raster.crs)
gdf_q2.to_file(join(mod_sfincs.root, 'gis', f'src_snapped_q2{postfix}.geojson'), driver='GeoJSON')